In [1]:
#patching
import torch, os, sys, itertools, glob
from pathlib import Path
import json, datetime, yaml, pickle
import numpy as np
from scipy.optimize import linear_sum_assignment 

NB_ROOT = "/mnt/nas_siddharth/code_final/notebooks"
os.chdir(NB_ROOT) 

CODE_ROOT = "/mnt/nas_siddharth/code_final"
sys.path.append(CODE_ROOT)
os.chdir(CODE_ROOT)

from src.pipeline import step1_proc2d as step1

FREE_DEVICE = "cuda:0"
step1.DEVICE = FREE_DEVICE
step1.BOTSORT_CFG["device"] = torch.device(FREE_DEVICE)


In [ ]:
from src.pipeline import (
    step2_crossviewmatching as step2,
    step3_crossframematching as step3,
    step4_aniposefiltering as step4,
)

FPS               = 24.0
N_KP              = 17
RESULTS_SWEEP_DIR = "/mnt/nas_siddharth/code_final/results3D_sweep"
RAW_DATA_ROOT     = "/mnt/nas_siddharth/tracking_gt/rawdata"
CALIB_FILE        = "/mnt/nas_siddharth/tracking_gt/calib/config.yaml"

BUF_FRAMES = 100

SESSIONS  = [
    "gp1d_20211015_170000",
    "gp1d_20211016_170000",
    "gp1d_20211017_170000",
    "gp1d_20211019_170000",
]

def kf_to_sec(kf):  
    return kf / 2.5 + 5.0

KF_RANGE_MAP = {
    "gp1d_20211015_170000": (5900 - BUF_FRAMES, 6650 + BUF_FRAMES),
    "gp1d_20211016_170000": (3050 - BUF_FRAMES, 3800 + BUF_FRAMES),
    "gp1d_20211017_170000": (   0 - BUF_FRAMES,  750 + BUF_FRAMES),
    "gp1d_20211019_170000": (3150 - BUF_FRAMES, 3900 + BUF_FRAMES),
}

SECOND_RANGES = [
    ( int(kf_to_sec(5800)),  int(kf_to_sec(6750)) ),   # 2365–2705s
    ( int(kf_to_sec(2950)),  int(kf_to_sec(3900)) ),   # 1185–1565s
    ( 0,                     int(kf_to_sec( 850)) ),   # 0–345s
    ( int(kf_to_sec(3050)),  int(kf_to_sec(4000)) ),   # 1225–1605s
]


#match_thresh_list      = [0.65, 0.75, 0.85]
#proximity_thresh_list  = [0.30, 0.40, 0.50, 0.60]
#track_low_thresh_list  = [0.05, 0.15, 0.25, 0.35]
#track_buffer_list      = [36, 72, 120]
#fuse_first_list        = [False, True]
#
#SWEEP_GRID = list(itertools.product(
#    match_thresh_list,
#    proximity_thresh_list,
#    track_low_thresh_list,
#    track_buffer_list,
#    fuse_first_list,
#))

SWEEP_GRID = [
    (0.65, 0.30, 0.05,  36,  False),
    (0.65, 0.30, 0.25, 120,  False),
    (0.85, 0.50, 0.05,  36,  True),
    (0.85, 0.50, 0.25, 120,  True),
]

print(f"{len(SWEEP_GRID)=}")



len(SWEEP_GRID)=4


In [3]:
LOG_CSV = os.path.join(RESULTS_SWEEP_DIR, "sweep_log.csv")
os.makedirs(RESULTS_SWEEP_DIR, exist_ok=True) 
if not os.path.exists(LOG_CSV):
    with open(LOG_CSV, "w") as f:
        f.write("session,match,prox,tlow,tbuf,fuse,results_root,tp,fn,fp,precision,recall\n")

def write_global_log(session, params, root, extra=""):
    m, prx, tl, tb, fu = params
    line = f"{session},{m},{prx},{tl},{tb},{fu},{root}"
    if extra:
        line += f",{extra}"
    with open(LOG_CSV, "a") as f:
        f.write(line + "\n")

def write_local_meta(root, session, params):
    m, prx, tl, tb, fu = params
    meta = {
        "session": session,
        "params": {
            "match_thresh": m,
            "proximity_thresh": prx,
            "track_low_thresh": tl,
            "track_buffer": tb,
            "fuse_first_associate": fu,
            "track_high_thresh": 0.85,
            "new_track_thresh": 0.60,
        },
        "timestamp": datetime.datetime.now().isoformat(),
    }
    Path(root).mkdir(parents=True, exist_ok=True)
    with open(os.path.join(root, "sweep_meta.json"), "w") as f:
        json.dump(meta, f, indent=2)


In [6]:
def build_results_root(params, session):
    m, prx, tl, tb, fu = params
    param_tag = f"MT{m:.2f}_PR{prx:.2f}_TL{tl:.2f}_TB{tb}" + ("_FF" if fu else "")
    return os.path.join(RESULTS_SWEEP_DIR, session, param_tag)

def patch_botsort_cfg(p):
    m, prx, tl, tb, fu = p
    cfg = step1.BOTSORT_CFG
    cfg.update({
        "match_thresh": m,
        "proximity_thresh": prx,
        "track_low_thresh": tl,
        "track_buffer": tb,
        "fuse_first_associate": fu,
        "track_high_thresh": 0.85,
        "new_track_thresh": 0.60,
    })


In [ ]:
for (session, (start_sec, end_sec)) in zip(SESSIONS, SECOND_RANGES):
    t_intv = (start_sec, end_sec)

    for params in SWEEP_GRID:
        results_root = build_results_root(params, session)
        os.makedirs(results_root, exist_ok=True)
        range_tag = f"{start_sec:04d}-{end_sec:04d}"
        data_name_range = f"{session}.{range_tag}"

        write_local_meta(results_root, session, params)

        print("-"*80)
        print(f"SESSION {session}  PARAMS {params}")
        print("-"*80)

        done_flag = os.path.join(results_root, f"{session}_DONE.flag")
        key_output_folder = os.path.join(results_root, data_name_range)
        key_output_file = os.path.join(key_output_folder, "kp3d.pickle")
        if os.path.exists(done_flag) or os.path.exists(key_output_file):
            print(f"[skip] {session}  {params} already processed")
            write_global_log(session, params, results_root)
            continue  # next param

        # Step-1
        patch_botsort_cfg(params)
        step1.step1_proc2d_custom(
            data_name   = session,
            results_root= results_root,
            raw_root    = RAW_DATA_ROOT,
            fps         = FPS,
            t_intv      = t_intv,
            redo        = False,
        )

        # Step-2
        step2.proc(
            data_name        = data_name_range,
            result_dir_root  = results_root,
            raw_data_dir     = RAW_DATA_ROOT,
            config_path      = CALIB_FILE,
            show_result      = False,
        )

        # Step-3
        step3.proc(
            data_name        = data_name_range,
            result_dir_root  = results_root,
            raw_data_dir     = RAW_DATA_ROOT,
            config_path      = CALIB_FILE,
            save_vid         = False,
        )

        # Step-4
        step4.proc(
            data_name        = data_name_range,
            results_dir_root = results_root,
            config_path      = CALIB_FILE,
            n_kp             = N_KP,
            redo             = True,
        )

        write_global_log(session, params, results_root,
                         extra=f"{tp},{fn},{fp},{prec:.3f},{rec:.3f}")

        Path(done_flag).touch()


--------------------------------------------------------------------------------
SESSION gp1d_20211015_170000  PARAMS (0.65, 0.3, 0.05, 36, False)
--------------------------------------------------------------------------------
[skip] gp1d_20211015_170000  (0.65, 0.3, 0.05, 36, False) already processed
--------------------------------------------------------------------------------
SESSION gp1d_20211015_170000  PARAMS (0.65, 0.3, 0.25, 120, False)
--------------------------------------------------------------------------------
[skip] gp1d_20211015_170000  (0.65, 0.3, 0.25, 120, False) already processed
--------------------------------------------------------------------------------
SESSION gp1d_20211015_170000  PARAMS (0.85, 0.5, 0.05, 36, True)
--------------------------------------------------------------------------------
06/13 10:50:57 - mmengine - INFO - [INFO] Initializing models on device: cuda:0
Loads checkpoint by local backend from path: ./model/detection/detection.pth
Loads 

gp1d_20211015_170000.22972495:   0%|          | 0/9120 [00:00<?, ?it/s]

06/13 10:51:20 - mmengine - WARNING - The current default scope "mmpretrain" is not "mmpose", `init_default_scope` will force set the currentdefault scope to "mmpose".


gp1d_20211015_170000.22972495:   9%|▉         | 822/9120 [02:40<33:55,  4.08it/s]

In [ ]:
# outputs

import glob, pandas as pd
rows = []
for res_dir in sorted(glob.glob(os.path.join(RESULTS_SWEEP_DIR, "TH*"))):
    stats_file = glob.glob(os.path.join(res_dir, "**", "kp3d.pickle"), recursive=True)
    rows.append({
        "run"    : os.path.basename(res_dir),
        "ok"     : len(stats_file) > 0,
        "sizeMB" : sum(os.path.getsize(p) for p in stats_file) / 1e6,
    })
pd.DataFrame(rows)


In [ ]:
import pandas as pd
pd.read_csv(os.path.join(RESULTS_SWEEP_DIR, "sweep_log.csv")).tail()